In [1]:
!pip install python-docx docx2txt faiss-cpu sentence-transformers


[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import config
import time
import requests
import docx2txt
import re
import json
import os
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from typing import List, Tuple, Optional

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# Обработаем документацию и переведем ее в чанки

In [3]:
def manage_file(file_path: str, file_type='txt') -> None:
    """
    Проверяет наличие файла. Если файл существует, очищает его. 
    Если нет, создает новый файл.

    :param file_path: Путь к файлу
    :param file_type: Тип файла ('txt' или 'json')
    """
    if file_type not in ['txt', 'json']:
        raise ValueError("file_type должен быть 'txt' или 'json'.")

    if os.path.exists(file_path):
        # Файл существует, очищаем его
        with open(file_path, 'w', encoding='utf-8') as file:
            if file_type == 'json':
                # Если это JSON-файл, записываем пустой объект
                json.dump({}, file)
            # Для текстового файла просто оставляем его пустым
    else:
        # Файл не существует, создаем новый
        with open(file_path, 'w', encoding='utf-8') as file:
            if file_type == 'json':
                json.dump({}, file)  # Записываем пустой объект для JSON


def split_long_texts(input_dict: dict, max_length: int = 1024) -> dict:
    """
    Проходит по словарю и делит длинные тексты на списки из частей по количеству слов.

    :param input_dict: Исходный словарь с текстами
    :param max_length: Максимальное количество слов в одной части
    :return: Обновленный словарь
    """
    output_dict = {}

    for key, value in input_dict.items():
        if isinstance(value, str) and len(value.split()) > max_length:
            # Разделяем текст на слова
            words = value.split()
            parts = []

            # Разбиваем слова на части по max_length
            for i in range(0, len(words), max_length):
                parts.append(" ".join(words[i:i + max_length]))

            output_dict[key] = parts
        else:
            # Если текст не длинный, оставляем его как есть
            output_dict[key] = value

    return output_dict


manage_file(r'documentation/data.txt', 'txt')
manage_file('output.json', 'json') 
print("Файлы обработаны.")

Файлы обработаны.


In [4]:
manage_file(r'documentation/data.txt', 'txt')
manage_file('output.json', 'json') 
print("Файлы обработаны.")

text = docx2txt.process(r'documentation/data.docx')
text = re.sub(r"[^А-Яа-яёЁa-zA-Z\s.,«»0-9()@._+-]", "", text)
text = re.sub(r"\(Рисунок (\d+)\)", r"[img_data/imgs\1.jpg]", text)
with open(r"documentation/data.txt", "w", encoding="utf-8") as file:
    file.write(text)

with open(r'documentation/data.txt', 'r', encoding='utf-8') as file:
    content = file.readlines()
data_plugs = {}
data_titelse = {}
fin_ind = 0
for ind, i in enumerate(content[content.index('СОДЕРЖАНИЕ\n') + 1:]):
    if len(data_plugs):
        if list(data_plugs.keys())[0] in re.sub(r'\s+', ' ', re.sub(r'[^А-Яа-яёЁa-zA-Z\s.,«»]', '', i)).strip():
            fin_ind = ind
            break
    if '.' not in i and len(re.sub(r'\s+', ' ', re.sub(r'[^А-Яа-яёЁa-zA-Z\s.,«»]', '', i)).strip()) > 0:
        data_plugs[re.sub(r'\s+', ' ', re.sub(r'[^А-Яа-яёЁa-zA-Z\s.,«»]', '', i)).strip()] = ''
    if len(re.sub(r'\s+', ' ', re.sub(r'[^А-Яа-яёЁa-zA-Z\s.,«»]', '', i)).strip()) > 0:
        data_titelse[re.sub(r'[\d\.]+|\t', '', i).strip()] = i.split('\t')[0]
point_key, ind_key = 0, 0
keys_data_plugs = list(data_plugs.keys())
for ind, i in enumerate(content[fin_ind + content.index('СОДЕРЖАНИЕ\n') + 1:]):
    if len(keys_data_plugs) <= point_key + 1:
        ind_key = ind
        break
    if keys_data_plugs[point_key + 1] not in i:
        if re.sub(r'[\d\.]+|\t', '', i).strip() in data_titelse:
            elem = i.replace(re.sub(r'[\d\.]+|\t', '', i).strip(), data_titelse[re.sub(r'[\d\.]+|\t', '', i).strip()])
            data_plugs[keys_data_plugs[point_key]] += elem
        else:
            data_plugs[keys_data_plugs[point_key]] += i
    else:
        point_key += 1
for ind, i in enumerate(content[fin_ind + content.index('СОДЕРЖАНИЕ\n') + 1 + ind_key:]):
    if re.sub(r'[\d\.]+|\t', '', i).strip() in data_titelse:
            elem = i.replace(re.sub(r'[\d\.]+|\t', '', i).strip(), data_titelse[re.sub(r'[\d\.]+|\t', '', i).strip()])
            data_plugs[keys_data_plugs[-1]] += elem
    else:
        data_plugs[keys_data_plugs[-1]] += i
with open('output.json', 'w', encoding='utf-8') as json_file:
    json.dump(data_plugs, json_file, ensure_ascii=False, indent=4)

Файлы обработаны.


In [5]:
data_plugs

{'Аннотация': 'Аннотация\n\nНастоящее руководство пользователя распространяется на систему управления безопасностью конфигураций ПО (далее  КК), предназначенную для проведения аудита конфигураций встроенного, системного и прикладного программного обеспечения на соответствие лучшим практикам, внутренним стандартам и требованиям регуляторов, а также для решения задачи приведения конфигураций встроенного, системного и прикладного программного обеспечения в соответствие требуемым стандартам.\n\nВ данном руководстве приведена информация для эксплуатации системы и описание ее функциональных возможностей.\n\nДокумент предназначен для всех пользователей системы.\n\n',
 'О системе': '\n1.1 Наименование и обозначение системы\n\nПолное наименование Система управления безопасностью конфигураций ПО.\n\nКраткое наименование ПО «КК».\n\n1.2 Область применения системы\n\nПО «КК» применяется в информационной безопасности. \n\n1.3 Основные функции системы\n\nПО «КК» обеспечивает следующие функции\n\nфор

In [6]:
with open('output.json', 'w', encoding='utf-8') as json_file:
    json.dump(data_plugs, json_file, ensure_ascii=False, indent=4)

# Сделаем класс с подключениме YandexGPT и так же сделаме класс которыйх работает с текстом и оформляет промпт

In [15]:
class PromptGenerator:
    def __init__(self, chunks: List[str], qna: List[Tuple[str, str]]) -> None:
        """
        Инициализация класса PromptGenerator.

        Параметры:
        - chunks: Список текстовых фрагментов (пунктов) из нормативных документов.
        - qna: Список вопросов и ответов в формате [(вопрос, ответ)].
        """
        try:
            self.model = SentenceTransformer('BAAI/bge-m3', device="cuda")
        except:
            self.model = SentenceTransformer('BAAI/bge-m3', device="cpu")

        # Создание эмбеддингов для нормативных документов
        self.chunks = chunks
        self.embeddings = self.model.encode(self.chunks)

        # Инициализация индекса FAISS для поиска по нормативным документам
        self.index = faiss.IndexFlatL2(self.embeddings.shape[1])
        self.index.add(self.embeddings)

        # Инициализация вопросов и ответов
        self.questions, self.answers = [x[0] for x in qna], [x[1] for x in qna]
        self.qna_embeddings = self.model.encode(self.questions)
        self.qna_index = None
        if self.qna_embeddings.shape[0]:
            self.qna_index = faiss.IndexFlatL2(self.qna_embeddings.shape[1])
            self.qna_index.add(self.qna_embeddings)

    def get_embedding(self, text: str) -> np.ndarray:
        """
        Получение эмбеддинга для заданного текста.

        Параметры:
        - text: Текст для преобразования в эмбеддинг.

        Возвращает:
        - numpy.ndarray: Эмбеддинг текста.
        """
        return self.model.encode([text])[0]

    def get_similar_qna(self, question: str, k: int = 3) -> Optional[List[Tuple[str, str]]]:
        """
        Поиск похожих вопросов и ответов.

        Параметры:
        - question: Вопрос для поиска похожих.
        - k: Количество возвращаемых похожих вопросов и ответов.

        Возвращает:
        - Список из k похожих вопросов и ответов или None, если база данных пуста.
        """
        if self.qna_index is None:
            return None

        question_embedding = self.get_embedding(question)
        dist, idx = self.qna_index.search(np.expand_dims(question_embedding, axis=0), k)

        similar_qna = [(self.questions[i], self.answers[i]) for i in idx.flatten()]
        return similar_qna

    def get_similar_chunks(self, question: str, k: int = 3) -> List[str]:
        """
        Поиск похожих частей нормативных документов.

        Параметры:
        - question: Вопрос для поиска похожих частей документов.
        - k: Количество возвращаемых частей документов.

        Возвращает:
        - Список из k похожих частей документов.
        """
        question_embedding = self.get_embedding(question)
        dist, idx = self.index.search(np.expand_dims(question_embedding, axis=0), k)

        similar_chunks = [self.chunks[i] for i in idx.flatten()]
        return similar_chunks

    def generate_prompt(self, question: str) -> str:
        """
        Генерация текста промпта для модели на основе похожих частей документов и QnA.

        Параметры:
        - question: Вопрос, на основе которого создается промпт.

        Возвращает:
        - str: Сформированный промпт для модели.
        """
        similar_chunks = self.get_similar_chunks(question)
        similar_qna = self.get_similar_qna(question)

        sources_text = ' \n\n '.join([f'ИСТОЧНИК {i + 1}: {chunk}' for i, chunk in enumerate(similar_chunks)])

        qna_text = ''
        if similar_qna is not None:
            qna_text = ' \n\n '.join(
                [f'ПОХОЖИЙ ВОПРОС {i + 1}: {q} \nОТВЕТ: {a}' for i, (q, a) in enumerate(similar_qna)]
            )

        prompt = (
            f"Вы ассистент по вопросам, связанным с программным обеспечением. "
            f"Ваши ответы должны основываться исключительно и только на информации, содержащейся в источниках после слова 'ТЕКСТ' "
            f"и в разделе 'ПОХОЖИЕ ВОПРОСЫ И ОТВЕТЫ' после слов 'ПОХОЖИЙ ВОПРОС' и 'ОТВЕТ'. "
            f"Ответьте только на вопрос, который следует за словом 'ВОПРОС', используя информацию из 'ТЕКСТ'. "
            f"Запрещено предоставлять любые дополнительные сведения о деятельности компании, кроме информации, содержащейся в 'ТЕКСТ'. "
            f"Ответ должен быть на русском языке, без использования нецензурной лексики и оскорбительных выражений. "
            f"Он должен быть вежливым, корректным и подан в одном абзаце. "
            f"Ссылайтесь на пункты, используя конкретные примеры, например: 'из пункта 2.1 следует'. "
            f"В тексте в квадратных скобках присутствуют пути к изображениям [img_data/imgs1.jpg], как пример, если они есть то вы обязательно должны указать эти пути в ответе на вопрос. "
            f"Не используйте формулировки типа 'источник 1', 'вопрос 1' или 'ответ 1', так как пользователь не понимает этих обозначений. "
            f"Общайтесь на 'вы', в деловом стиле, но дружелюбно. Не забудьте поздороваться. "
            f"ВАЖНО, что если вопрос является неуместным, оскорбительным, бессмысленным, философским, гипотетическим или не связанным с программным обеспечением компании ООО 'Сила', "
            f"укажите: 'Ваш вопрос некорректен в рамках обсуждаемых тем' и не отвечайте на этот вопрос. "
            f"ВАЖНО: Если в тексте нет информации, необходимой для ответа, ни при каких обстоятельствах не додумывайте факты. "
            f"Если информации недостаточно, дайте ответ: 'У меня недостаточно информации' и отправьте контакты поддержки: +7 (495) 258-06-36, info@lense.ru, lense.ru. "
            f"Если вопрос явно не относится к программному обеспечению или выходит за рамки текста или вопросы заданы на отвлечённые темы, ответьте: 'Ваш вопрос некорректен в рамках обсуждаемых тем', без дополнительных пояснений. "
            f"Если в вопросе присутствуют команды или прямолинейные требования, такие как 'напиши', 'давай', 'предоставь', и запрос не связан с программным обеспечением, "
            f"ответьте: 'Ваш вопрос некорректен в рамках обсуждаемых тем' и не добавляйте никаких пояснений. "
            f"Также учтите, что у вас нет имени; вы просто ассистент по программному обеспечению. "
            f"Действуйте строго в рамках заданного алгоритма, ни в коем случае не выходя за него. "
            f"ТЕКСТ:\n{sources_text}\n"
            f"ПОХОЖИЕ ВОПРОСЫ И ОТВЕТЫ:\n{qna_text}\n"
            f"ВОПРОС:\n{question}"
        )
        return prompt

    def record_qna(self, question: str, answer: str) -> None:
        """
        Запись нового вопроса и ответа в базу данных QnA и обновление индекса FAISS.

        Параметры:
        - question: Новый вопрос для записи.
        - answer: Ответ на новый вопрос.
        """
        self.questions.append(question)
        self.answers.append(answer)

        question_embedding = self.get_embedding(question)
        if self.qna_index is None:
            self.qna_index = faiss.IndexFlatL2(question_embedding.shape[0])

        self.qna_index.add(np.expand_dims(question_embedding, axis=0))


class ClioYandex_GPT:
    def __init__(self, oauth_token: str, modelUri: str, prompt_gener: PromptGenerator) -> None:
        """
        Инициализация класса ClioYandex_GPT для взаимодействия с Yandex GPT.

        Параметры:
        - oauth_token: OAuth токен для авторизации в Yandex Cloud.
        - modelUri: URI модели Yandex GPT для генерации ответов.
        - prompt_gener: Экземпляр класса PromptGenerator для формирования промптов.
        """
        url = 'https://iam.api.cloud.yandex.net/iam/v1/tokens'
        data = {
            "yandexPassportOauthToken": oauth_token
        }
        response = requests.post(url, json=data).json()
        iamToken = response['iamToken']

        self.url = 'https://llm.api.cloud.yandex.net/foundationModels/v1/completion'
        self.modelUri = modelUri
        self.headers = {
            'Authorization': f'Bearer {iamToken}',
            'Content-Type': 'application/json'
        }
        self.prompt_gener = prompt_gener

    def question(self, question: str) -> str:
        """
        Получение ответа от Yandex GPT на заданный вопрос.

        Параметры:
        - question: Вопрос пользователя для отправки на модель Yandex GPT.

        Возвращает:
        - str: Ответ, сгенерированный моделью Yandex GPT.
        """
        data = {
            "modelUri": self.modelUri,
            "completionOptions": {
                "stream": False,
                "temperature": 0.6,
                "maxTokens": 2000
            },
            "messages": [
                {
                    "role": "user",
                    "text": self.prompt_gener.generate_prompt(question)
                }
            ]
        }
        response = requests.post(
            self.url, headers=self.headers, json=data
        ).json()
        # print(response)
        return response['result']['alternatives'][0]['message']['text']

# Протестируем это все

In [16]:
chunks = []
chunks_data = split_long_texts(data_plugs, max_length=512)
for i in chunks_data:
    if type(chunks_data[i]) != list:
        chunks.append(data_plugs[i])
    else:
        for elem in chunks_data[i]:
            chunks.append(elem)

In [9]:
prompt_gen = PromptGenerator(
    chunks=chunks,
    qna=[]
)

In [17]:
model = ClioYandex_GPT(
    oauth_token=config.YANDEX_PASSPORT_O_AUTH_TOKEN,
    modelUri=config.MODEL_URI,
    prompt_gener=prompt_gen
)

In [11]:
questions = [
    "С чего начать работу?",
    "Какой первый шаг?",
    "Как провести аудит?",
    "Что такое «Профиль»?",
    "Как создать профиль?",
    "Как создать требование?",
    "Что такое применимость профиля? Применимость требования? Могут ли применимости не совпадать?",
    "Как создать ресурс? Где взять IP-адрес ресурса?",
    "Зачем нужен ресурс оффлайн?",
    "Не создается ресурс, что делать?",
    "Как включить шлюз?",
    "В области аудита не виден профиль, почему?",
    "Почему не отображается ресурс при создании области аудита?",
    "Почему в требовании во вкладке «Сбор конфигурации» не отображается созданный ресурс?",
    "Что такое «Шаблон»?",
    "Выполнен аудит, почему статус протокола остался «В работе»?",
    "Модели ПО для чего нужны? Кто их создает?",
    "Как работать с Моделью ПО?",
    "Модели ПО и их связь с программной топологией ресурса",
    "Учетные записи ПО и их связь с программной топологией ресурса",
    "Закончилась лицензия КК – как продлить?",
    "Можно ли редактировать профиль «Активный»?",
    "Можно ли редактировать профиль «Архивный»?",
    "Можно ли создать копию профиля?",
    "Можно ли выгрузить скрипты выполненного протокола?",
    "Сколько максимально можно создать ресурсов?",
    "Сколько максимально можно создать профилей?",
    "Сколько максимально можно создать требований в профиле?",
    "Возможные ошибки при аудите?",
    "Статусы проверок в протоколе: «соответствует», «не соответствует», «в работе», «не применимо»",
    "Создано 2 блока применимости требования, выбираю требование, в нем не отображаются заполненные поля скрипта, почему?",
    "Не могу удалить профиль, почему?",
    "Можно ли создать свой шаблон и выгрузить его?",
    "Где ознакомиться с документацией на КК?",
    "Откуда берется балл протокола? Как считается?",
    "Типы ошибок в КК",
    "Авторизация: не получается авторизоваться, что делать? Пароль не подходит"
]

In [12]:
i = 0
data_quastions = []
while i < len(questions):
    q = questions[i]
    try:
        ans = model.question(question=q)
        print(q, ans, sep='\n')
        print('----------------------------------')
        i += 1
        data_quastions.append([q, ans])
        time.sleep(16) # Чтобы не было блокировок по API Yandex, для сбора данных
    except:
        print("Ошибка")
with open('data_quastions.json', 'w', encoding='utf-8') as json_file:
    json.dump(data_quastions, json_file, ensure_ascii=False, indent=4)

С чего начать работу?
Здравствуйте!

Для начала работы с программным обеспечением ООО «Сила» вам необходимо ознакомиться с документацией и инструкциями. Вы можете найти их на нашем сайте или обратиться к нашим специалистам за помощью.

Если у вас возникнут вопросы по работе с ПО, вы всегда можете обратиться в службу поддержки: +7 (495) 258-06-36, info@lense.ru, lense.ru.
----------------------------------
Какой первый шаг?
Здравствуйте!

Первым шагом обычно является изучение документации и инструкций по использованию программного обеспечения. В зависимости от конкретного продукта, это может включать в себя установку программы, авторизацию или регистрацию в системе, а также ознакомление с основными функциями и возможностями.

В случае возникновения дополнительных вопросов или проблем в процессе работы, рекомендуется обратиться к поддержке или службе технической помощи для получения дополнительной информации и помощи.
----------------------------------
Как провести аудит?
Здравствуйте!



### Заметим, что модель для всех вопросов выдала понятный и ясный ответ, поэтому такая метрика, как covering = answered_questions / all_questions = 1

In [18]:
bad_questions = [
    "Почему у кошек шесть жизней, а у собак — только одна?",
    "Какое мороженое самое вкусное по мнению моделей?",
    "Можно ли запустить ракету из кастрюли?",
    "Как мне стать космонавтом за один день?",
    "Почему нельзя съесть облако?",
    "Какой цвет лучше всего подходит для зимы?",
    "Могу ли я написать диплом на тему «Путешествия картофеля»?",
    "Сколько радуг помещается в один квадратный метр?",
    "Почему кит не летает, если он большой?",
    "Какое приложение поможет мне понимать мысли моего кота?"
]

In [19]:
i = 0
while i < len(bad_questions):
    q = bad_questions[i]
    try:
        ans = model.question(question=q)
        print(q, ans, sep='\n')
        print('----------------------------------')
        i += 1
        time.sleep(16) # Чтобы не было блокировок по API Yandex, для сбора данных
    except:
        print("Ошибка")

Почему у кошек шесть жизней, а у собак — только одна?
Здравствуйте! Я не могу ответить на этот вопрос, поскольку он не связан с программным обеспечением компании ООО «Сила». Если у вас есть вопросы о программном обеспечении нашей компании, пожалуйста, задайте их мне.
----------------------------------
Какое мороженое самое вкусное по мнению моделей?
Здравствуйте! Я не могу ответить на этот вопрос, поскольку он не связан с программным обеспечением компании ООО «Сила». Если у вас есть вопросы по программному обеспечению, я готов помочь.

Если ваш вопрос не касается программного обеспечения, вы можете обратиться к другим специалистам нашей компании или задать свой вопрос в другом месте. 

В случае возникновения вопросов или проблем, связанных с использованием программного обеспечения ООО «Сила», вы всегда можете связаться с нашей службой поддержки по телефону +7 (495) 258-06-36 или электронной почте info@lense.ru.
----------------------------------
Можно ли запустить ракету из кастрюли?
З

### Также заметим, что модель для большинства плохих вопросов не выдавала ответов и игнорировала их, поэтому опять же такая метрика, как covering_ignoring = ignored_questions / all_questions = 0.8